# 텍스트 분석
출처 - teanaps 패키지 : https://github.com/fingeredman/teanaps

In [ ]:
## 필요한 패키지 설치 8분내외 걸림

# 나눔고딕 설치
# !apt-get update -qq
# !apt-get install fonts-nanum* -qq
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

# 텍스트 마이닝 패키지 teanaps 설치
!git clone https://github.com/ByungjunKim/teanaps.git  # 경량화 버전(분석에 필요한 패키지만 설치) made by 김병준
!python "teanaps/teanaps_setup.py"

# 패키지 설치
!pip install -U udkanbun gdown gensim -q

In [ ]:
# # 아래 코드 실행 후 한글 폰트가 깨지면 '런타임' 메뉴에서 '런타임 다시시작'
# import matplotlib.pyplot as plt
# import numpy as np
# import matplotlib as mpl
# # 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
# mpl.rcParams['axes.unicode_minus'] = False
# plt.rc('font', family='NanumBarunGothic')
# font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# import matplotlib.colors as clr

# # plot 한글 깨짐 확인용 -> 한글 깨지면 런타임 재시작 (런타임 메뉴 -> 런타임 다시 시작 후 다시 셀 실행)
# data = np.random.randint(-100, 100, 50).cumsum()
# plt.plot(range(50), data, 'r')
# # mpl.rcParams['axes.unicode_minus'] = False
# plt.title('시간별 가격 추이')
# plt.ylabel('주식 가격')
# plt.xlabel('시간(분)')

In [ ]:
import udkanbun
lzh=udkanbun.load()

import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()

from collections import Counter
import itertools

import nltk
from nltk import collocations

# teanaps
from teanaps.text_analysis import TfidfCalculator #빈도분석
from teanaps.text_analysis import CoWordCalculator #공기어 분석

from gensim.models import Word2Vec
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import matplotlib as mpl
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
import matplotlib.colors as clr

### 데이터 전처리

##### 분석할 데이터 로드

In [ ]:
# 참고 : https://hansonminlearning.tistory.com/34
# https://docs.google.com/spreadsheets/d/13EXU26CO71UVzGiymbRed4jqT7HhrbHI/
!gdown --id 13EXU26CO71UVzGiymbRed4jqT7HhrbHI --output analects_sent.xlsx # 논어 데이터를 쓰고 싶으면 실행

In [ ]:
df = pd.read_excel('analects_sent.xlsx') # 분석하길 원하는 파일로 설정
df

In [ ]:
# 대화문(sentence)이 없는 행 삭제
df = df.dropna(subset=['sentence']) #'sentence' 열을 기준으로 결측행 삭제
df

In [ ]:
# reset index
df = df.reset_index(drop=True)
df

##### 형태소 분석
명사, 대명사, 동사, 부사, 형용사만 추출

In [ ]:
# 원하는 품사 태그를 포함해 토크나이징
def tokenize_tag(sentence,allow_pos=[]):
  s = lzh(sentence)
  if allow_pos !=[]:
    res = [t.form+'/'+t.upos.lower() for t in s if t.upos in allow_pos]
  else:
    res = [t.form+'/'+t.upos.lower() for t in s]
  return res

In [ ]:
# 원하는 품사 태그를 미포함해 토크나이징
def tokenize(sentence,allow_pos=[]):
  s = lzh(sentence)
  if allow_pos !=[]:
    res = [t.form for t in s if t.upos in allow_pos]
  else:
    res = [t.form for t in s]
  return res

In [ ]:
# 토크나이징
df['token'] = df['sentence'].progress_map(lambda x:tokenize(x,['NOUN','PROPN','VERB','ADV', 'ADJ'])) # 품사 미포함
df['token_tag'] = df['sentence'].progress_map(lambda x:tokenize_tag(x,['NOUN','PROPN','VERB','ADV', 'ADJ'])) # 품사 포함

In [ ]:
# 결과 확인
df[['sentence','token','token_tag']]

##### 불용어 & 동의어 처리

In [ ]:
# 불용어 리스트
stopwords = ['不/adv'] # 품사를 포함한 경우 '不/ADV' 이렇게 써야함(형태소/품사)

df['token_tag'] = df['token_tag'].map(lambda x:[w for w in x if not w in set(stopwords)])

In [ ]:
df['token_tag']

### 빈도 분석

In [ ]:
# 상위 n개 단어와 빈도 확인
cnt = Counter(list(itertools.chain(*df['token_tag'].tolist())))
cnt.most_common(50) # 상위 N개

In [ ]:
# 빈도 분석 함수 활성화
tfidf = TfidfCalculator()

In [ ]:
# teanaps 용 리스트로 변환
token_tag_list = df['token_tag'].map(lambda x: ' '.join(x)).tolist()

In [ ]:
# 빈도 상위 n개 단어만 tfidf 학습
tfidf.calculation_tfidf(token_tag_list, tfidf_count=100, tfidf_state=True)

In [ ]:
# 문장-단어 단순 빈도 행렬
tfidf.get_tf_matrix()

In [ ]:
# 문장 - 단어 TFIDF 행렬
tfidf.get_tfidf_matrix()

In [ ]:
# 상위 n개 단어 (TF, TF-IDF)
tf_df = pd.DataFrame(tfidf.get_tf_list()[:100], columns = ['keywords_tf','tf'])
tfidf_df = pd.DataFrame(tfidf.get_tfidf_list()[:100],columns = ['keywords_tfidf','tfidf'])
tf_tfidf_df = pd.concat([tf_df,tfidf_df],axis=1)
tf_tfidf_df

In [ ]:
# tf, tf-idf 키워드 빈도 엑셀로 저장
tf_tfidf_df.to_excel('tf_tfidf_df.xlsx', index = None)

In [ ]:
# 단어 빈도 시각화
tfidf.draw_tfidf(100)

### 단어 동시출현 분석 (공기어, 共起語, co-occurance)

In [ ]:
# 동시출현 분석 함수활성화
co = CoWordCalculator()

In [ ]:
# tfidf 기준 상위 100개단어
node_list = tf_tfidf_df['keywords_tfidf'].tolist()

In [ ]:
# 공기어 함수에 분석할 node_list 투입
co.calculation_co_matrix(token_tag_list,node_list=node_list)

In [ ]:
# edge
co.get_edge_list()

In [ ]:
# node
co.get_node_list()

In [ ]:
# 특정 단어를 기준으로 다른단어들과의 동시출현 빈도 계산
co.get_co_word('仁/noun')

In [ ]:
# 네트워크 중심성(centrality) 계산 - degree
co.get_centrality("d_cent")

In [ ]:
# 네트워크 중심성(centrality) 계산 - betweeness
co.get_centrality("b_cent")

In [ ]:
# 네트워크 중심성(centrality) 계산 - closeness
co.get_centrality("c_cent")

In [ ]:
# 동시출현 빈도 매트릭스 시각화
co.get_co_matrix_graph(25) # 상위 n개 단어

In [ ]:
# 단어 네트워크 시각화 (markers)
co.get_word_network_graph(co.get_centrality("d_cent"), mode="markers") # degree 기준

In [ ]:
# 단어 네트워크 시각화(text)
co.get_word_network_graph(co.get_centrality("d_cent"), mode="text") # degree 기준

In [ ]:
# 단어 네트워크 시각화 (marker + text)
co.get_word_network_graph(co.get_centrality("d_cent"), mode="markers+text") # degree 기준

In [ ]:
# betweeness 기준?


In [ ]:
# export edge table for Gephi
edge_table = pd.DataFrame([(a,b,c) for (a,b), c in co.get_edge_list()], columns= ['Source','Target','Weight'])
edge_table

In [ ]:
edge_table.to_csv('edge_table.csv',index=None)

### word2vec 

##### word2vec 학습

In [ ]:
# vector : 차원, window : 앞뒤 단어 보는 크기, min_count : 분석에 포함할 최소 단어 빈도수, sg : 0이면 cbow, 1이면 skip-gram
w2v = Word2Vec(sentences=df['token_tag'].to_list(), vector_size=300, window=3, min_count=3,sg=1,seed=2021, workers=1)

##### 특정 단어와 문맥이 비슷한 유사어 추출

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
w2v.wv.most_similar('仁/noun', topn=20)  # 상위 n개

In [ ]:
# 해당 단어(2개 이상)와 문맥이 비슷한 유사어 추출
w2v.wv.most_similar(['仁/noun','義/noun'], topn=20)

In [ ]:
# 단어끼리 계산 해보기 
# 仁 + 義 - 天 =?
w2v.wv.most_similar(positive=['仁/noun','義/noun'],negative=['天/noun'],topn=20) 

In [ ]:
# 입력한 단어 중 가장 관련성이 적은 것을 뺀다면?
w2v.wv.doesnt_match(['仁/noun','義/noun','天/noun'])

##### wor2vec 2차원 시각화

In [ ]:
#단어 리스트 생성
vocab =list(w2v.wv.key_to_index)
X = w2v.wv[vocab]

In [ ]:
# 단어 - 차원
w2v.wv[vocab].shape

In [ ]:
tsne = TSNE(n_components=2) # 2차원으로 축소

In [ ]:
X_tsne = tsne.fit_transform(X[:50,:]) # 빈도수 상위 50개
X_tsne

In [ ]:
# 차원축소 결과 확인
w2v_50 = pd.DataFrame(X_tsne, index=vocab[:50], columns=['x', 'y'])
w2v_50

In [ ]:
# word2vec 상위 50개 단어 시각화
fig = plt.figure()
fig.set_size_inches(20, 12)
ax = fig.add_subplot(1, 1, 1)

ax.scatter(w2v_50['x'], w2v_50['y'])

for word, pos in w2v_50.iterrows():
    ax.annotate(word, pos)
plt.show()